<a href="https://colab.research.google.com/github/Gleicyelle/AssistenteOAB/blob/main/AssistenteOAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
!pip install -U -q google-generativeai

In [129]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("SECRET_KEY")
G_API_KEY = api_key
genai.configure(api_key = G_API_KEY)

In [130]:
#encontrar modelo de embedding
for a in genai.list_models():
  if 'embedContent' in a.supported_generation_methods:
    embedding_model = a.name
    break

if not embedding_model:
    raise ValueError("Nenhum modelo de embedding encontrado!")

In [131]:
#exemplo de embedding
title = "a"
sample_text = ("ab")

embeddings = genai.embed_content(model = "models/embedding-001",
                                 content = sample_text,
                                 title = title,
                                 task_type = "RETRIEVAL_DOCUMENT")

In [132]:
#carregar o assunto
doc = {
    "Título": "CÓDIGO DE ÉTICA E DISCIPLINA DA OAB",
    "Conteúdo": "O Conselho Federal da Ordem dos Advogados do Brasil, ao instituir o Código de Ética e Disciplina, norteou-se por princípios que formam a consciência profissional do advogado e representam imperativos de sua conduta, os quais se traduzem nos seguintes mandamentos: lutar sem receio pelo primado da Justiça; pugnar pelo cumprimento da Constituição e pelo respeito à Lei, fazendo com que o ordenamento jurídico seja interpretado com retidão, em perfeita sintonia com os fins sociais a que se dirige e as exigências do bem comum; ser fiel à verdade para poder servir à Justiça como um de seus elementos essenciais; proceder com lealdade e boa-fé em suas relações profissionais e em todos os atos do seu ofício; empenhar-se na defesa das causas confiadas ao seu patrocínio, dando ao constituinte o amparo do Direito, e proporcionando-lhe a realização prática de seus legítimos interesses; comportar-se, nesse mister, com independência e altivez, defendendo com o mesmo denodo humildes e poderosos; exercer a advocacia com o indispensável senso profissional, mas também com desprendimento, jamais permitindo que o anseio de ganho material sobreleve a finalidade social do seu trabalho; aprimorar-se no culto dos princípios éticos e no domínio da ciência jurídica, de modo a tornar-se merecedor da confiança do cliente e da sociedade como um todo, pelos atributos intelectuais e pela probidade pessoal; agir, em suma, com a dignidade e a correção dos profissionais que honram e engrandecem a sua classe. Inspirado nesses postulados, o Conselho Federal da Ordem dos Advogados do Brasil, no uso das atribuições que lhe são conferidas pelos arts. 33 e 54, V, da Lei n. 8.906, de 04 de julho de 1994, aprova e edita este Código, exortando os advogados brasileiros à sua fiel observância."}
doc1 = {
    "Título": "CAPÍTULO I (TÍTULO I) - DOS PRINCÍPIOS FUNDAMENTAIS",
    "Conteúdo": "Art. 1º O exercício da advocacia exige conduta compatível com os preceitos deste Código, do Estatuto, do Regulamento Geral, dos Provimentos e com os princípios da moral individual, social e profissional. Art. 2º O advogado, indispensável à administração da Justiça, é defensor do Estado Democrático de Direito, dos direitos humanos e garantias fundamentais, da cidadania, da moralidade, da Justiça e da paz social, cumprindo-lhe exercer o seu ministério em consonância com a sua elevada função pública e com os valores que lhe são inerentes. São deveres do advogado: I – preservar, em sua conduta, a honra, a nobreza e a dignidade da profissão, zelando pelo caráter de essencialidade e indispensabilidade da advocacia; II – atuar com destemor, independência, honestidade, decoro, veracidade, lealdade, dignidade e boa-fé; III – velar por sua reputação pessoal e profissional; IV – empenhar-se, permanentemente, no aperfeiçoamento pessoal e profissional; V – contribuir para o aprimoramento das instituições, do Direito e das leis; VI – estimular, a qualquer tempo, a conciliação e a mediação entre os litigantes, prevenindo, sempre que possível, a instauração de litígios; VII – desaconselhar lides temerárias, a partir de um juízo preliminar de viabilidade jurídica; VIII – abster-se de: a) utilizar de influência indevida, em seu benefício ou do cliente; b) vincular seu nome ou nome social a empreendimentos sabidamente escusos; (NR) c) emprestar concurso aos que atentem contra a ética, a moral, a honestidade e a dignidade da pessoa humana; d) entender-se diretamente com a parte adversa que tenha patrono constituído, sem o assentimento deste; e) ingressar ou atuar em pleitos administrativos ou judiciais perante autoridades com as quais tenha vínculos negociais ou familiares; f) contratar honorários advocatícios em valores aviltantes. IX – pugnar pela solução dos problemas da cidadania e pela efetivação dos direitos individuais, coletivos e difusos; X – adotar conduta consentânea com o papel de elemento indispensável à administração da Justiça; XI – cumprir os encargos assumidos no âmbito da Ordem dos Advogados do Brasil ou na representação da classe; XII – zelar pelos valores institucionais da OAB e da advocacia; XIII – ater-se, quando no exercício da função de defensor público, à defesa dos necessitados. Art. 3º O advogado deve ter consciência de que o Direito é um meio de mitigar as desigualdades para o encontro de soluções justas e que a lei é um instrumento para garantir a igualdade de todos. Art. 4º O advogado, ainda que vinculado ao cliente ou constituinte, mediante relação empregatícia ou por contrato de prestação permanente de serviços, ou como integrante de departamento jurídico, ou de órgão de assessoria jurídica, público ou privado, deve zelar pela sua liberdade e independência. É legítima a recusa, pelo advogado, do patrocínio de causa e de manifestação, no âmbito consultivo, de pretensão concernente a direito que também lhe seja aplicável ou contrarie orientação que tenha manifestado anteriormente. Art. 5º O exercício da advocacia é incompatível com qualquer procedimento de mercantilização. Art. 6º É defeso ao advogado expor os fatos em Juízo ou na via administrativa falseando deliberadamente a verdade e utilizando de má-fé. Art. 7º É vedado o oferecimento de serviços profissionais que implique, direta ou indiretamente, angariar ou captar clientela."}
doc2 = {
    "Título": "CAPÍTULO II (TÍTULO I) - DA ADVOCACIA PÚBLICA",
    "Conteúdo": "Art. 8º As disposições deste Código obrigam igualmente os órgãos de advocacia pública, e advogados públicos, incluindo aqueles que ocupem posição de chefia e direção jurídica. § 1º O advogado público exercerá suas funções com independência técnica, contribuindo para a solução ou redução de litigiosidade, sempre que possível. § 2º O advogado público, inclusive o que exerce cargo de chefia ou direção jurídica, observará nas relações com os colegas, autoridades, servidores e o público em geral, o dever de urbanidade, tratando a todos com respeito e consideração, ao mesmo tempo em que preservará suas prerrogativas e o direito de receber igual tratamento das pessoas com as quais se relacione."}
doc3 = {
    "Título": "CAPÍTULO III (TÍTULO I) - DAS RELAÇÕES COM O CLIENTE",
    "Conteúdo": "Art. 9º O advogado deve informar o cliente, de modo claro e inequívoco, quanto a eventuais riscos da sua pretensão, e das consequências que poderão advir da demanda. Deve, igualmente, denunciar, desde logo, a quem lhe solicite parecer ou patrocínio, qualquer circunstância que possa influir na resolução de submeter-lhe a consulta ou confiar-lhe a causa. Art. 10. As relações entre advogado e cliente baseiam-se na confiança recíproca. Sentindo o advogado que essa confiança lhe falta, é recomendável que externe ao cliente sua impressão e, não se dissipando as dúvidas existentes, promova, em seguida, o substabelecimento do mandato ou a ele renuncie. Art. 11. O advogado, no exercício do mandato, atua como patrono da parte, cumprindo-lhe, por isso, imprimir à causa orientação que lhe pareça mais adequada, sem se subordinar a intenções contrárias do cliente, mas, antes, procurando esclarecê-lo quanto à estratégia traçada. Art. 12. A conclusão ou desistência da causa, tenha havido, ou não, extinção do mandato, obriga o advogado a devolver ao cliente bens, valores e documentos que lhe hajam sido confiados e ainda estejam em seu poder, bem como a prestar-lhe contas, pormenorizadamente, sem prejuízo de esclarecimentos complementares que se mostrem pertinentes e necessários. A parcela dos honorários paga pelos serviços até então prestados não se inclui entre os valores a ser devolvidos. Art. 13. Concluída a causa ou arquivado o processo, presume-se cumprido e extinto o mandato. Art. 14. O advogado não deve aceitar procuração de quem já tenha patrono constituído, sem prévio conhecimento deste, salvo por motivo plenamente justificável ou para adoção de medidas judiciais urgentes e inadiáveis. Art. 15. O advogado não deve deixar ao abandono ou ao desamparo as causas sob seu patrocínio, sendo recomendável que, em face de dificuldades insuperáveis ou inércia do cliente quanto a providências que lhe tenham sido solicitadas, renuncie ao mandato. Art. 16. A renúncia ao patrocínio deve ser feita sem menção do motivo que a determinou, fazendo cessar a responsabilidade profissional pelo acompanhamento da causa, uma vez decorrido o prazo previsto em lei (EAOAB, Art. 5º, § 3º). § 1º A renúncia ao mandato não exclui responsabilidade por danos eventualmente causados ao cliente ou a terceiros. § 2º O advogado não será responsabilizado por omissão do cliente quanto a documento ou informação que lhe devesse fornecer para a prática oportuna de ato processual do seu interesse. Art. 17. A revogação do mandato judicial por vontade do cliente não o desobriga do pagamento das verbas honorárias contratadas, assim como não retira o direito do advogado de receber o quanto lhe seja devido em eventual verba honorária de sucumbência, calculada proporcionalmente em face do serviço efetivamente prestado. Art. 18. O mandato judicial ou extrajudicial não se extingue pelo decurso de tempo, salvo se o contrário for consignado no respectivo instrumento. Art. 19. Os advogados integrantes da mesma sociedade profissional, ou reunidos em caráter permanente para cooperação recíproca, não podem representar, em juízo ou fora dele, clientes com interesses opostos. Art. 20. Sobrevindo conflito de interesses entre seus constituintes e não conseguindo o advogado harmonizá-los, caber-lhe-á optar, com prudência e discrição, por um dos mandatos, renunciando aos demais, resguardado sempre o sigilo profissional. Art. 21. O advogado, ao postular em nome de terceiros, contra ex-cliente ou ex- empregador, judicial e extrajudicialmente, deve resguardar o sigilo profissional. Art. 22. Ao advogado cumpre abster-se de patrocinar causa contrária à validade ou legitimidade de ato jurídico em cuja formação haja colaborado ou intervindo de qualquer maneira; da mesma forma, deve declinar seu impedimento ou o da sociedade que integre quando houver conflito de interesses motivado por intervenção anterior no trato de assunto que se prenda ao patrocínio solicitado. Art. 23. É direito e dever do advogado assumir a defesa criminal, sem considerar sua própria opinião sobre a culpa do acusado. Não há causa criminal indigna de defesa, cumprindo ao advogado agir, como defensor, no sentido de que a todos seja concedido tratamento condizente com a dignidade da pessoa humana, sob a égide das garantias constitucionais. Art. 24. O advogado não se sujeita à imposição do cliente que pretenda ver com ele atuando outros advogados, nem fica na contingência de aceitar a indicação de outro profissional para com ele trabalhar no processo. Art. 25. É defeso ao advogado funcionar no mesmo processo, simultaneamente, como patrono e preposto do empregador ou cliente. Art. 26. O substabelecimento do mandato, com reserva de poderes, é ato pessoal do advogado da causa. § 1º O substabelecimento do mandato sem reserva de poderes exige o prévio e inequívoco conhecimento do cliente. § 2º O substabelecido com reserva de poderes deve ajustar antecipadamente seus honorários com o substabelecente."}
doc4 = {
    "Título": "CAPÍTULO IV (TÍTULO I) - DAS RELAÇÕES COM OS COLEGAS, AGENTES POLÍTICOS, AUTORIDADES, SERVIDORES PÚBLICOS E TERCEIROS",
    "Conteúdo": "Art. 27. O advogado observará, nas suas relações com os colegas de profissão, agentes políticos, autoridades, servidores públicos e terceiros em geral, o dever de urbanidade, tratando a todos com respeito e consideração, ao mesmo tempo em que preservará seus direitos e prerrogativas, devendo exigir igual tratamento de todos com quem se relacione. § 1º O dever de urbanidade há de ser observado, da mesma forma, nos atos e manifestações relacionados aos pleitos eleitorais no âmbito da Ordem dos Advogados do Brasil. § 2º No caso de ofensa à honra do advogado ou à imagem da instituição, adotar-se- ão as medidas cabíveis, instaurando-se processo ético-disciplinar e dando-se ciência às autoridades competentes para apuração de eventual ilícito penal. Art. 28. Consideram-se imperativos de uma correta atuação profissional o emprego de linguagem escorreita e polida, bem como a observância da boa técnica jurídica. Art. 29. O advogado que se valer do concurso de colegas na prestação de serviços advocatícios, seja em caráter individual, seja no âmbito de sociedade de advogados ou de empresa ou entidade em que trabalhe, dispensar-lhes-á tratamento condigno, que não os torne subalternos seus nem lhes avilte os serviços prestados mediante remuneração incompatível com a natureza do trabalho profissional ou inferior ao mínimo fixado pela Tabela de Honorários que for aplicável. Quando o aviltamento de honorários for praticado por empresas ou entidades públicas ou privadas, os advogados responsáveis pelo respectivo departamento ou gerência jurídica serão instados a corrigir o abuso, inclusive intervindo junto aos demais órgãos competentes e com poder de decisão da pessoa jurídica de que se trate, sem prejuízo das providências que a Ordem dos Advogados do Brasil possa adotar com o mesmo objetivo."}
doc5 = {
    "Título": "CAPÍTULO V (TÍTULO I) - DA ADVOCACIA PRO BONO",
    "Conteúdo": "Art. 30. No exercício da advocacia pro bono, e ao atuar como defensor nomeado, conveniado ou dativo, o advogado empregará o zelo e a dedicação habituais, de forma que a parte por ele assistida se sinta amparada e confie no seu patrocínio. § 1º Considera-se advocacia pro bono a prestação gratuita, eventual e voluntária de serviços jurídicos em favor de instituições sociais sem fins econômicos e aos seus assistidos, sempre que os beneficiários não dispuserem de recursos para a contratação de profissional. § 2º A advocacia pro bono pode ser exercida em favor de pessoas naturais que, igualmente, não dispuserem de recursos para, sem prejuízo do próprio sustento, contratar advogado. § 3º A advocacia pro bono não pode ser utilizada para fins político-partidários ou eleitorais, nem beneficiar instituições que visem a tais objetivos, ou como instrumento de publicidade para captação de clientela."}
doc6 = {
    "Título": "CAPÍTULO VI (TÍTULO I) - DO EXERCÍCIO DE CARGOS E FUNÇÕES NA OAB E NA REPRESENTAÇÃO DA CLASSE",
    "Conteúdo": "Art. 31. O advogado, no exercício de cargos ou funções em órgãos da Ordem dos Advogados do Brasil ou na representação da classe junto a quaisquer instituições, órgãos ou comissões, públicos ou privados, manterá conduta consentânea com as disposições deste Código e que revele plena lealdade aos interesses, direitos e prerrogativas da classe dos advogados que representa. Art. 32. Não poderá o advogado, enquanto exercer cargos ou funções em órgãos da OAB ou representar a classe junto a quaisquer instituições, órgãos ou comissões, públicos ou privados, firmar contrato oneroso de prestação de serviços ou fornecimento de produtos com tais entidades nem adquirir bens imóveis ou móveis infungíveis de quaisquer órgãos da OAB, ou a estes aliená-los. Não há impedimento ao exercício remunerado de atividade de magistério na Escola Nacional de Advocacia – ENA, nas Escolas de Advocacia – ESAs e nas Bancas do Exame de Ordem, observados os princípios da moralidade e da modicidade dos valores estabelecidos a título de remuneração. (NR) Art. 33. Salvo em causa própria, não poderá o advogado, enquanto exercer cargos ou funções em órgãos da OAB ou tiver assento, em qualquer condição, nos seus Conselhos, atuar em processos que tramitem perante a entidade nem oferecer pareceres destinados a instruí-los. A vedação estabelecida neste artigo não se aplica aos dirigentes de Seccionais quando atuem, nessa qualidade, como legitimados a recorrer nos processos em trâmite perante os órgãos da OAB. Art. 34. Ao submeter seu nome à apreciação do Conselho Federal ou dos Conselhos Seccionais com vistas à inclusão em listas destinadas ao provimento de vagas reservadas à classe nos tribunais, no Conselho Nacional de Justiça, no Conselho Nacional do Ministério Público e em outros colegiados, o candidato assumirá o compromisso de respeitar os direitos e prerrogativas do advogado, não praticar nepotismo nem agir em desacordo com a moralidade administrativa e com os princípios deste Código, no exercício de seu mister."}
doc7 = {
    "Título": "CAPÍTULO VII (TÍTULO I) - DO SIGILO PROFISSIONAL",
    "Conteúdo": "Art. 35. O advogado tem o dever de guardar sigilo dos fatos de que tome conhecimento no exercício da profissão. O sigilo profissional abrange os fatos de que o advogado tenha tido conhecimento em virtude de funções desempenhadas na Ordem dos Advogados do Brasil. Art. 36. O sigilo profissional é de ordem pública, independendo de solicitação de reserva que lhe seja feita pelo cliente. § 1º Presumem-se confidenciais as comunicações de qualquer natureza entre advogado e cliente. § 2º O advogado, quando no exercício das funções de mediador, conciliador e árbitro, se submete às regras de sigilo profissional. Art. 37. O sigilo profissional cederá em face de circunstâncias excepcionais que configurem justa causa, como nos casos de grave ameaça ao direito à vida e à honra ou que envolvam defesa própria. Art. 38. O advogado não é obrigado a depor, em processo ou procedimento judicial, administrativo ou arbitral, sobre fatos a cujo respeito deva guardar sigilo profissional."}
doc8 = {
    "Título": "CAPÍTULO VIII (TÍTULO I) - DA PUBLICIDADE PROFISSIONAL",
    "Conteúdo": "Art. 39. A publicidade profissional do advogado tem caráter meramente informativo e deve primar pela discrição e sobriedade, não podendo configurar captação de clientela ou mercantilização da profissão. Art. 40. Os meios utilizados para a publicidade profissional hão de ser compatíveis com a diretriz estabelecida no artigo anterior, sendo vedados: I – a veiculação da publicidade por meio de rádio, cinema e televisão; II – o uso de outdoors, painéis luminosos ou formas assemelhadas de publicidade; III – as inscrições em muros, paredes, veículos, elevadores ou em qualquer espaço público; IV – a divulgação de serviços de advocacia juntamente com a de outras atividades ou a indicação de vínculos entre uns e outras; V – o fornecimento de dados de contato, como endereço e telefone, em colunas ou artigos literários, culturais, acadêmicos ou jurídicos, publicados na imprensa, bem assim quando de eventual participação em programas de rádio ou televisão, ou em veiculação de matérias pela internet, sendo permitida a referência a e-mail; VI – a utilização de mala direta, a distribuição de panfletos ou formas assemelhadas de publicidade, com o intuito de captação de clientela. Exclusivamente para fins de identificação dos escritórios de advocacia, é permitida a utilização de placas, painéis luminosos e inscrições em suas fachadas, desde que respeitadas as diretrizes previstas no artigo 39. Art. 41. As colunas que o advogado mantiver nos meios de comunicação social ou os textos que por meio deles divulgar não deverão induzir o leitor a litigar nem promover, dessa forma, captação de clientela. Art. 42. É vedado ao advogado: I – responder com habitualidade a consulta sobre matéria jurídica, nos meios de comunicação social; II – debater, em qualquer meio de comunicação, causa sob o patrocínio de outro advogado; III – abordar tema de modo a comprometer a dignidade da profissão e da instituição que o congrega; IV – divulgar ou deixar que sejam divulgadas listas de clientes e demandas; V – insinuar-se para reportagens e declarações públicas. Art. 43. O advogado que eventualmente participar de programa de televisão ou de rádio, de entrevista na imprensa, de reportagem televisionada ou veiculada por qualquer outro meio, para manifestação profissional, deve visar a objetivos exclusivamente ilustrativos, educacionais e instrutivos, sem propósito de promoção pessoal ou profissional, vedados pronunciamentos sobre métodos de trabalho usados por seus colegas de profissão. Quando convidado para manifestação pública, por qualquer modo e forma, visando ao esclarecimento de tema jurídico de interesse geral, deve o advogado evitar insinuações com o sentido de promoção pessoal ou profissional, bem como o debate de caráter sensacionalista. Art. 44. Na publicidade profissional que promover ou nos cartões e material de escritório de que se utilizar, o advogado fará constar seu nome, nome social ou o da sociedade de advogados, o número ou os números de inscrição na OAB. (NR) § 1º Poderão ser referidos apenas os títulos acadêmicos do advogado e as distinções honoríficas relacionadas à vida profissional, bem como as instituições jurídicas de que faça parte, e as especialidades a que se dedicar, o endereço, e-mail, site, página eletrônica, QR code, logotipo e a fotografia do escritório, o horário de atendimento e os idiomas em que o cliente poderá ser atendido. § 2º É vedada a inclusão de fotografias pessoais ou de terceiros nos cartões de visitas do advogado, bem como menção a qualquer emprego, cargo ou função ocupado, atual ou pretérito, em qualquer órgão ou instituição, salvo o de professor universitário. Art. 45. São admissíveis como formas de publicidade o patrocínio de eventos ou publicações de caráter científico ou cultural, assim como a divulgação de boletins, por meio físico ou eletrônico, sobre matéria cultural de interesse dos advogados, desde que sua circulação fique adstrita a clientes e a interessados do meio jurídico. Art. 46. A publicidade veiculada pela internet ou por outros meios eletrônicos deverá observar as diretrizes estabelecidas neste capítulo. A telefonia e a internet podem ser utilizadas como veículo de publicidade, inclusive para o envio de mensagens a destinatários certos, desde que estas não impliquem o oferecimento de serviços ou representem forma de captação de clientela. Art. 47. As normas sobre publicidade profissional constantes deste capítulo poderão ser complementadas por outras que o Conselho Federal aprovar, observadas as diretrizes do presente Código. Art. 47-A. Será admitida a celebração de termo de ajustamento de conduta no âmbito dos Conselhos Seccionais e do Conselho Federal para fazer cessar a publicidade irregular praticada por advogados e estagiários. (NR) O termo previsto neste artigo será regulamentado mediante edição de provimento do Conselho Federal, que estabelecerá seus requisitos e condições. (NR)"}
doc9 = {
    "Título": "CAPÍTULO IX (TÍTULO I) - DOS HONORÁRIOS PROFISSIONAIS",
    "Conteúdo": "Art. 48. A prestação de serviços profissionais por advogado, individualmente ou integrado em sociedades, será contratada, preferentemente, por escrito. § 1º O contrato de prestação de serviços de advocacia não exige forma especial, devendo estabelecer, porém, com clareza e precisão, o seu objeto, os honorários ajustados, a forma de pagamento, a extensão do patrocínio, esclarecendo se este abrangerá todos os atos do processo ou limitar-se-á a determinado grau de jurisdição, além de dispor sobre a hipótese de a causa encerrar-se mediante transação ou acordo. § 2º A compensação de créditos, pelo advogado, de importâncias devidas ao cliente, somente será admissível quando o contrato de prestação de serviços a autorizar ou quando houver autorização especial do cliente para esse fim, por este firmada. § 3º O contrato de prestação de serviços poderá dispor sobre a forma de contratação de profissionais para serviços auxiliares, bem como sobre o pagamento de custas e emolumentos, os quais, na ausência de disposição em contrário, presumem-se devam ser atendidos pelo cliente. Caso o contrato preveja que o advogado antecipe tais despesas, ser-lhe-á lícito reter o respectivo valor atualizado, no ato de prestação de contas, mediante comprovação documental. § 4º As disposições deste capítulo aplicam-se à mediação, à conciliação, à arbitragem ou a qualquer outro método adequado de solução dos conflitos. § 5º É vedada, em qualquer hipótese, a diminuição dos honorários contratados em decorrência da solução do litígio por qualquer mecanismo adequado de solução extrajudicial. § 6º Deverá o advogado observar o valor mínimo da Tabela de Honorários instituída pelo respectivo Conselho Seccional onde for realizado o serviço, inclusive aquele referente às diligências, sob pena de caracterizar-se aviltamento de honorários. § 7º O advogado promoverá, preferentemente, de forma destacada a execução dos honorários contratuais ou sucumbenciais. Art. 49. Os honorários profissionais devem ser fixados com moderação, atendidos os elementos seguintes: I – a relevância, o vulto, a complexidade e a dificuldade das questões versadas; II – o trabalho e o tempo a ser empregados; III – a possibilidade de ficar o advogado impedido de intervir em outros casos, ou de se desavir com outros clientes ou terceiros; IV – o valor da causa, a condição econômica do cliente e o proveito para este resultante do serviço profissional; V – o caráter da intervenção, conforme se trate de serviço a cliente eventual, frequente ou constante; VI – o lugar da prestação dos serviços, conforme se trate do domicílio do advogado ou de outro; VII – a competência do profissional; VIII – a praxe do foro sobre trabalhos análogos. Art. 50. Na hipótese da adoção de cláusula quota litis, os honorários devem ser necessariamente representados por pecúnia e, quando acrescidos dos honorários da sucumbência, não podem ser superiores às vantagens advindas a favor do cliente. § 1º A participação do advogado em bens particulares do cliente só é admitida em caráter excepcional, quando esse, comprovadamente, não tiver condições pecuniárias de satisfazer o débito de honorários e ajustar com o seu patrono, em instrumento contratual, tal forma de pagamento. § 2º Quando o objeto do serviço jurídico versar sobre prestações vencidas e vincendas, os honorários advocatícios poderão incidir sobre o valor de umas e outras, atendidos os requisitos da moderação e da razoabilidade. Art. 51. Os honorários da sucumbência e os honorários contratuais, pertencendo ao advogado que houver atuado na causa, poderão ser por ele executados, assistindo-lhe direito autônomo para promover a execução do capítulo da sentença que os estabelecer ou para postular, quando for o caso, a expedição de precatório ou requisição de pequeno valor em seu favor. § 1º No caso de substabelecimento, a verba correspondente aos honorários da sucumbência será repartida entre o substabelecente e o substabelecido, proporcionalmente à atuação de cada um no processo ou conforme haja sido entre eles ajustado. § 2º Quando for o caso, a Ordem dos Advogados do Brasil ou os seus Tribunais de Ética e Disciplina poderão ser solicitados a indicar mediador que contribua no sentido de que a distribuição dos honorários da sucumbência, entre advogados, se faça segundo o critério estabelecido no § 1o. § 3º Nos processos disciplinares que envolverem divergência sobre a percepção de honorários da sucumbência, entre advogados, deverá ser tentada a conciliação destes, preliminarmente, pelo relator. Art. 52. O crédito por honorários advocatícios, seja do advogado autônomo, seja de sociedade de advogados, não autoriza o saque de duplicatas ou qualquer outro título de crédito de natureza mercantil, podendo, apenas, ser emitida fatura, quando o cliente assim pretender, com fundamento no contrato de prestação de serviços, a qual, porém, não poderá ser levada a protesto. Pode, todavia, ser levado a protesto o cheque ou a nota promissória emitido pelo cliente em favor do advogado, depois de frustrada a tentativa de recebimento amigável. Art. 53. É lícito ao advogado ou à sociedade de advogados empregar, para o recebimento de honorários, sistema de cartão de crédito, mediante credenciamento junto a empresa operadora do ramo. Eventuais ajustes com a empresa operadora que impliquem pagamento antecipado não afetarão a responsabilidade do advogado perante o cliente, em caso de rescisão do contrato de prestação de serviços, devendo ser observadas as disposições deste quanto à hipótese. Art. 54. Havendo necessidade de promover arbitramento ou cobrança judicial de honorários, deve o advogado renunciar previamente ao mandato que recebera do cliente em débito."}

documents = [doc, doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9]

In [133]:
df = pd.DataFrame(documents)
df

,Título,Conteúdo
0,CÓDIGO DE ÉTICA E DISCIPLINA DA OAB,O Conselho Federal da Ordem dos Advogados do B...
1,CAPÍTULO I (TÍTULO I) - DOS PRINCÍPIOS FUNDAME...,Art. 1º O exercício da advocacia exige conduta...
2,CAPÍTULO II (TÍTULO I) - DA ADVOCACIA PÚBLICA,Art. 8º As disposições deste Código obrigam ig...
3,CAPÍTULO III (TÍTULO I) - DAS RELAÇÕES COM O C...,"Art. 9º O advogado deve informar o cliente, de..."
4,CAPÍTULO IV (TÍTULO I) - DAS RELAÇÕES COM OS C...,"Art. 27. O advogado observará, nas suas relaçõ..."
5,CAPÍTULO V (TÍTULO I) - DA ADVOCACIA PRO BONO,"Art. 30. No exercício da advocacia pro bono, e..."
6,CAPÍTULO VI (TÍTULO I) - DO EXERCÍCIO DE CARGO...,"Art. 31. O advogado, no exercício de cargos ou..."
7,CAPÍTULO VII (TÍTULO I) - DO SIGILO PROFISSIONAL,Art. 35. O advogado tem o dever de guardar sig...
8,CAPÍTULO VIII (TÍTULO I) - DA PUBLICIDADE PROF...,Art. 39. A publicidade profissional do advogad...
9,CAPÍTULO IX (TÍTULO I) - DOS HONORÁRIOS PROFIS...,Art. 48. A prestação de serviços profissionais...


In [135]:
def embed_fun(title, text):
    embedding = genai.embed_content(model=embedding_model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]
    return np.array(embedding)

In [136]:
#dataframe
df["embeddings"] = df.apply(lambda row: embed_fun(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,embeddings
0,CÓDIGO DE ÉTICA E DISCIPLINA DA OAB,O Conselho Federal da Ordem dos Advogados do B...,"[0.019368079, -0.018714162, -0.00976639, 0.010..."
1,CAPÍTULO I (TÍTULO I) - DOS PRINCÍPIOS FUNDAME...,Art. 1º O exercício da advocacia exige conduta...,"[0.001977729, -0.038513705, 0.015249065, 0.008..."
2,CAPÍTULO II (TÍTULO I) - DA ADVOCACIA PÚBLICA,Art. 8º As disposições deste Código obrigam ig...,"[0.032673962, -0.023955127, -0.01157058, -0.00..."
3,CAPÍTULO III (TÍTULO I) - DAS RELAÇÕES COM O C...,"Art. 9º O advogado deve informar o cliente, de...","[-0.00196135, -0.037708964, 0.000931858, -0.00..."
4,CAPÍTULO IV (TÍTULO I) - DAS RELAÇÕES COM OS C...,"Art. 27. O advogado observará, nas suas relaçõ...","[0.019969132, -0.03491194, -0.004895888, 0.025..."
5,CAPÍTULO V (TÍTULO I) - DA ADVOCACIA PRO BONO,"Art. 30. No exercício da advocacia pro bono, e...","[0.042857807, -0.0373357, -0.029759351, -0.009..."
6,CAPÍTULO VI (TÍTULO I) - DO EXERCÍCIO DE CARGO...,"Art. 31. O advogado, no exercício de cargos ou...","[-0.0109943, -0.04036449, 0.0003863819, 0.0270..."
7,CAPÍTULO VII (TÍTULO I) - DO SIGILO PROFISSIONAL,Art. 35. O advogado tem o dever de guardar sig...,"[0.01935991, -0.024178728, 0.0013257908, 0.014..."
8,CAPÍTULO VIII (TÍTULO I) - DA PUBLICIDADE PROF...,Art. 39. A publicidade profissional do advogad...,"[0.020061778, -0.06436921, 0.02136139, 0.00145..."
9,CAPÍTULO IX (TÍTULO I) - DOS HONORÁRIOS PROFIS...,Art. 48. A prestação de serviços profissionais...,"[0.004594394, -0.042537842, -0.0033150748, 0.0..."


In [137]:
#buscar respostas
def gerar_buscar_consulta(consulta, dataframe, model):
    embed_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]
    #converter a coluna "embeddings" para um array NumPy
    produtos_escalares = np.dot(np.array(dataframe["embeddings"].tolist()), embed_consulta)
    indice = np.argmax(produtos_escalares)
    return dataframe.iloc[indice]["Conteúdo"]

In [138]:
#especificações da IA
generation_config = {
    "temperature": 0.4,
    "candidate_count": 1
}

In [140]:
while True:
    print("CÓDIGOS DE ÉTICA E DISCIPLINA DA OAB")
    pergunta = input("Linha de consulta: ")
    if pergunta.lower() == 'sair':
        break

    trecho = gerar_buscar_consulta(pergunta, df, embedding_model)

    prompt = f"Responda a pergunta como se você fosse um especialista em direito dando uma explicação, sem adicionar informações que não façam parte do texto, faça em formato markdown: {trecho}"

    model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config = generation_config)
    response = model_2.generate_content(prompt)
    print("\n")
    print("--------")
    print(response.text)
    print("--------")
    print("\n")

CÓDIGOS DE ÉTICA E DISCIPLINA DA OAB
Linha de consulta: o que o advogado deve informar ao cliente?


--------
**Art. 9º**

O advogado tem o dever de informar o cliente sobre os riscos e consequências potenciais da demanda, de forma clara e inequívoca. Além disso, deve denunciar quaisquer circunstâncias que possam influenciar a decisão do cliente de contratar seus serviços.

**Art. 10**

As relações entre advogado e cliente baseiam-se na confiança mútua. Se o advogado sentir que essa confiança está comprometida, deve comunicar sua impressão ao cliente e, se as dúvidas persistirem, renunciar ou substabelecer o mandato.

**Art. 11**

O advogado atua como patrono da parte, orientando a causa conforme sua avaliação profissional. Ele não deve se subordinar a intenções contrárias do cliente, mas esclarecê-lo sobre a estratégia adotada.

**Art. 12**

Ao concluir ou desistir da causa, o advogado deve devolver ao cliente quaisquer bens, valores ou documentos confiados a ele, bem como prestar con